In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4 langchain-cohere

In [ ]:
# Import the needed libraries.
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
import getpass
import os

# Keep your API key secret.
os.environ["COHERE_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_cohere import ChatCohere

# You can choose any other model.
llm = ChatCohere(model="command-r")

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://www.spiceworks.com/tech/data-management/articles/what-is-kafka/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
from langchain_cohere import CohereEmbeddings

# We'll use CohereEmbeddings model.
embeddings_model = CohereEmbeddings()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

In [ ]:
# Retrieve and generate using the relevant snippets of the article.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Let's try our RAG.
rag_chain.invoke("What is Kafka?")

In [ ]:
# Clean our vectorstore after finishing.
vectorstore.delete_collection()